🔬 Chemometric Analysis Notebook
## FBMFOR — Food Fraud Analysis
**MSc Food Technology and Quality Assurance** | University of Reading

This notebook provides a complete chemometric analysis pipeline for spectroscopic data (NMR, FTIR) used in food fraud detection. You will work through:

1. **Data upload and exploration** — load your spectral data and metadata
2. **Preprocessing** — baseline correction, normalisation, and derivatives
3. **Principal Component Analysis (PCA)** — unsupervised exploration
4. **Hierarchical Cluster Analysis (HCA)** — sample grouping
5. **PLS-DA** — supervised classification
6. **OPLS-DA** — orthogonal signal correction for enhanced discrimination

Each section includes explanatory text describing the *what* and *why* of each step. You are encouraged to modify parameters and observe the effects.

> **Data format:** CSV files with samples as rows and spectral variables (wavenumbers/chemical shifts) as columns. The first column should contain sample IDs, and a separate metadata CSV maps sample IDs to class labels.

## 1. Setup and Installation

Run this cell to install and import all required packages. On Google Colab, most are pre-installed; only `plotly` may need updating.

In [ ]:
# ============================================================
# INSTALL & IMPORT
# ============================================================
# Uncomment the next line if plotly is outdated on your Colab instance
# !pip install --upgrade plotly

# --- Core numerical and data libraries ---
import numpy as np                  # Array operations, linear algebra
import pandas as pd                 # DataFrames for tabular data

# --- Plotting ---
import matplotlib.pyplot as plt     # Static plots (used for HCA dendrogram)
import plotly.express as px         # High-level interactive plots
import plotly.graph_objects as go   # Low-level interactive plot control
from plotly.subplots import make_subplots  # Multi-panel interactive figures

# --- Signal processing and clustering ---
from scipy import signal, sparse    # Savitzky-Golay filter, sparse matrices for ALS
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster  # HCA
from scipy.spatial.distance import pdist    # Pairwise distance computation
from scipy.stats import f as f_dist         # F-distribution for Hotelling's T²

# --- Machine learning ---
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Data scaling, label encoding
from sklearn.decomposition import PCA                           # Principal Component Analysis
from sklearn.cross_decomposition import PLSRegression           # PLS (used for PLS-DA and OPLS-DA)
from sklearn.model_selection import cross_val_predict, StratifiedKFold  # Cross-validation
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score  # Evaluation

# --- Utility ---
import warnings
warnings.filterwarnings("ignore")  # Suppress convergence warnings from PLS

print("✅ All packages imported successfully.")

## 2. Demo Dataset: Simulated Olive Oil FTIR

This section generates a **realistic simulated FTIR dataset** for olive oil authentication. The dataset mimics mid-infrared spectra (4000–400 cm⁻¹) for three classes:

- **Extra Virgin Olive Oil (EVOO)** — authentic
- **Refined Olive Oil (ROO)** — lower grade, sometimes mislabelled
- **Adulterated** — EVOO blended with hazelnut or sunflower oil

Key spectral differences are introduced in regions known to be diagnostic:
- ~1745 cm⁻¹ (carbonyl C=O stretch — ester linkage, differs with fatty acid profile)
- ~2925 and ~2854 cm⁻¹ (C–H stretching — related to chain length and saturation)
- ~1160 cm⁻¹ (C–O stretch — differs with triglyceride composition)
- ~3005 cm⁻¹ (=C–H stretch — indicator of unsaturation)

The simulation includes realistic sources of variation found in real laboratory data:
- **Peak position shifts** (±2–3 cm⁻¹) from instrument calibration and temperature drift
- **Intensity variation** (±15%) from natural compositional differences within each class
- **Sample-specific baselines** from different pathlengths and sample preparation
- **Three outlier samples** with intentional anomalies (borderline quality, ATR artefact, subtle adulteration) — see if you can find them with Hotelling’s T²!

> **Skip this section** if you want to upload your own data — proceed to Section 3.

In [ ]:
# ============================================================
# GENERATE SIMULATED OLIVE OIL FTIR DATA
# ============================================================
# Set the global random seed for reproducibility. All students
# running this notebook will get identical simulated spectra.
np.random.seed(11088)

# --- Define the wavenumber axis ---
# Mid-infrared range from 4000 to 400 cm-1, with 1800 points
# (approx. 2 cm-1 resolution, typical for benchtop FTIR)
wavenumbers = np.linspace(4000, 400, 1800)


def gaussian_peak(x, centre, width, height):
    """
    Generate a Gaussian absorption peak.
    
    Parameters:
        x      : array of wavenumber values
        centre : peak centre position (cm-1)
        width  : peak width at half-maximum (cm-1), controls broadness
        height : peak height (absorbance units)
    
    Returns:
        Array of absorbance values for this peak
    """
    return height * np.exp(-0.5 * ((x - centre) / width) ** 2)


def generate_spectrum(class_type, sample_idx):
    """
    Generate a simulated FTIR spectrum for a given oil class.
    
    Each sample has realistic individual variation controlled by
    a deterministic per-sample RNG (seeded from sample_idx), so
    the same sample_idx always produces the same spectrum.
    
    Sources of variation:
        - Peak position jitter (+/- ~3 cm-1): instrument/temperature drift
        - Intensity variation (+/- ~15%): natural compositional differences
        - Sample-specific baseline: pathlength and sample preparation
        - Three planted outlier samples for teaching outlier detection
    
    Parameters:
        class_type : str — "EVOO", "ROO", or "Adulterated"
        sample_idx : int — unique sample index (0-based), drives per-sample variation
    
    Returns:
        1D numpy array of simulated absorbance values
    """
    spectrum = np.zeros_like(wavenumbers)
    
    # --- Per-sample random number generator ---
    # Each sample gets its own RNG seeded deterministically from sample_idx,
    # ensuring reproducibility while giving each sample unique variation.
    # The multiplier 7 spaces seeds apart to reduce correlation.
    rng = np.random.RandomState(11088 + sample_idx * 7)
    
    # --- Per-sample variation parameters ---
    # Peak position jitter: sigma = 1.5 cm-1, so 95% of shifts are within +/- 3 cm-1
    # This simulates small instrument calibration differences between measurements
    pos_jitter = rng.normal(0, 1.5, 10)  # 10 values, one per peak
    
    # Intensity variation: sigma = 7%, so 95% of samples are within +/- 14%
    # This simulates natural compositional variation within each class
    # (e.g., not all EVOOs have identical oleic acid content)
    int_variation = 1.0 + rng.normal(0, 0.07, 10)
    
    # Sample-specific baseline: slope and offset
    # Simulates different ATR crystal contact, pathlength, or sample thickness
    baseline_slope = rng.normal(0, 0.003)
    baseline_offset = rng.normal(0, 0.015)
    
    # --- Common peaks present in ALL olive oil classes ---
    # These are the fundamental vibrational modes of triglycerides
    spectrum += gaussian_peak(wavenumbers, 2925 + pos_jitter[0], 30, 0.80 * int_variation[0])  # C-H asymmetric stretch (CH2)
    spectrum += gaussian_peak(wavenumbers, 2854 + pos_jitter[1], 25, 0.60 * int_variation[1])  # C-H symmetric stretch (CH2)
    spectrum += gaussian_peak(wavenumbers, 1745 + pos_jitter[2], 20, 0.90 * int_variation[2])  # C=O ester stretch (triglyceride backbone)
    spectrum += gaussian_peak(wavenumbers, 1465 + pos_jitter[3], 15, 0.35 * int_variation[3])  # C-H bending (scissoring)
    spectrum += gaussian_peak(wavenumbers, 1160 + pos_jitter[4], 25, 0.50 * int_variation[4])  # C-O stretch (ester linkage)
    spectrum += gaussian_peak(wavenumbers, 720  + pos_jitter[5], 12, 0.25 * int_variation[5])  # C-H rocking (long-chain alkyl)
    
    # --- Class-specific spectral differences ---
    # These reflect genuine chemical differences between oil types
    if class_type == "EVOO":
        # EVOO has high unsaturation (oleic, linoleic acid) and polyphenols
        spectrum += gaussian_peak(wavenumbers, 3005 + pos_jitter[6], 15, 0.30 * int_variation[6])  # Strong =C-H stretch (cis double bonds)
        spectrum += gaussian_peak(wavenumbers, 1650 + pos_jitter[7], 12, 0.08 * int_variation[7])  # Minor peak from polyphenol/water
    
    elif class_type == "ROO":
        # Refined oil: reduced unsaturation markers, altered ester profile
        spectrum += gaussian_peak(wavenumbers, 3005 + pos_jitter[6], 15, 0.18 * int_variation[6])  # Weaker =C-H (some unsaturation lost in refining)
        spectrum += gaussian_peak(wavenumbers, 1745 + pos_jitter[2], 20, 0.05 * int_variation[7])  # Slightly modified ester peak shape
    
    elif class_type == "Adulterated":
        # Adulterated with cheaper oils (sunflower, hazelnut): different fatty acid profile
        spectrum += gaussian_peak(wavenumbers, 3005 + pos_jitter[6], 15, 0.12 * int_variation[6])  # Much weaker =C-H (lower unsaturation type)
        spectrum += gaussian_peak(wavenumbers, 2925 + pos_jitter[0], 30, 0.10 * int_variation[7])  # Extra C-H intensity (sunflower oil contribution)
        spectrum += gaussian_peak(wavenumbers, 1160 + pos_jitter[4], 25, 0.08 * int_variation[8])  # Altered C-O profile (different triglycerides)
    
    # --- Outlier samples ---
    # Three samples have intentional anomalies to teach outlier detection.
    # Students should be able to identify these using Hotelling's T2.
    if sample_idx == 3:
        # EVOO_04: borderline quality — unusually low unsaturation with refined character.
        # This sample should appear between the EVOO and ROO clusters in PCA.
        spectrum += gaussian_peak(wavenumbers, 3005, 15, -0.12)  # Reduce the =C-H peak
        spectrum += gaussian_peak(wavenumbers, 1745, 20, 0.08)   # Stronger ester (more like ROO)
    
    elif sample_idx == 22:
        # ROO_11: ATR crystal artefact — a broad spurious bump around 2100 cm-1.
        # This is a common measurement error in ATR-FTIR and should be obvious in spectra.
        spectrum += gaussian_peak(wavenumbers, 2100, 80, 0.15)
    
    elif sample_idx == 31:
        # Adulterated_05: very subtle adulteration — hard to detect.
        # The =C-H peak is partially restored, making this sample overlap with ROO.
        spectrum += gaussian_peak(wavenumbers, 3005, 15, 0.10)
    
    # --- Measurement noise ---
    # White Gaussian noise with sigma = 0.012 absorbance units,
    # typical for a well-maintained benchtop FTIR instrument
    noise = rng.normal(0, 0.012, len(wavenumbers))
    
    # --- Baseline drift ---
    # Combination of a sinusoidal component (optical interference fringes)
    # and a linear component (sample-specific pathlength/preparation)
    baseline_drift = (0.02 * np.sin(wavenumbers / 800)
                      + baseline_slope * (wavenumbers - 2200) / 1800
                      + baseline_offset)
    
    spectrum += noise + baseline_drift
    return spectrum


# --- Generate the full dataset ---
# 15 EVOO + 12 ROO + 10 Adulterated = 37 samples total
# This is a realistic sample size for a teaching practical
classes = ["EVOO"] * 15 + ["ROO"] * 12 + ["Adulterated"] * 10
sample_ids = [f"{c}_{i+1:02d}" for i, c in enumerate(classes)]

# Generate all spectra as a 2D array (samples x wavenumbers)
spectra = np.array([generate_spectrum(c, i) for i, c in enumerate(classes)])

# --- Create pandas DataFrames ---
# Spectral data: columns are wavenumber values as strings (standard for spectral data)
spectral_columns = [f"{w:.1f}" for w in wavenumbers]
df_spectra = pd.DataFrame(spectra, columns=spectral_columns)
df_spectra.insert(0, "Sample_ID", sample_ids)  # First column is sample identifier

# Metadata: maps each sample to its class label
df_metadata = pd.DataFrame({"Sample_ID": sample_ids, "Class": classes})

# --- Report ---
print(f"Generated {len(df_spectra)} spectra with {len(wavenumbers)} wavenumber points.")
print(f"Classes: {dict(zip(*np.unique(classes, return_counts=True)))}")
print(f"\nSpectral data shape: {df_spectra.shape}")
print(f"Wavenumber range: {wavenumbers[-1]:.0f} – {wavenumbers[0]:.0f} cm⁻¹")
print(f"\nNote: Samples EVOO_04, ROO_11, and Adulterated_05 have intentional")
print(f"anomalies for outlier detection exercises.")

## 3. Data Upload (Your Own Data)

Use this section to upload your own spectral data. You need **two CSV files**:

1. **Spectral data** — rows = samples, columns = spectral variables (wavenumbers or chemical shifts). First column should be `Sample_ID`.
2. **Metadata** — at minimum two columns: `Sample_ID` and `Class`.

> **If using the demo dataset**, skip this cell — the data is already loaded above.

In [ ]:
# ============================================================
# UPLOAD YOUR OWN DATA (skip if using demo dataset)
# ============================================================
# Uncomment and run this cell to upload your own files.
# The Colab files.upload() widget will prompt you to select files
# from your local computer.

# from google.colab import files
#
# # --- Upload spectral data ---
# print("Upload your SPECTRAL DATA CSV:")
# uploaded = files.upload()                            # Opens file picker
# spectral_filename = list(uploaded.keys())[0]         # Get the uploaded filename
# df_spectra = pd.read_csv(spectral_filename)          # Parse CSV into DataFrame
#
# # --- Upload metadata ---
# print("\nUpload your METADATA CSV:")
# uploaded = files.upload()
# metadata_filename = list(uploaded.keys())[0]
# df_metadata = pd.read_csv(metadata_filename)
#
# # --- Verify upload ---
# print(f"\nSpectral data: {df_spectra.shape[0]} samples, {df_spectra.shape[1]-1} variables")
# print(f"Metadata: {df_metadata.shape[0]} samples, columns: {list(df_metadata.columns)}")
# print(f"Classes found: {df_metadata['Class'].value_counts().to_dict()}")

## 4. Data Exploration

Before any analysis, it is essential to **visually inspect your raw spectra**. Look for:
- Obvious outliers or failed measurements
- Baseline drift (common in FTIR)
- Differences in overall intensity between samples
- Key absorption bands that differ between groups

In [ ]:
# ============================================================
# PREPARE DATA MATRICES
# ============================================================
# Convert the DataFrames into numpy arrays suitable for analysis.
# X_raw will be the spectral matrix used throughout the notebook.

# Extract the numeric spectral data (drop the Sample_ID column)
X_raw = df_spectra.drop(columns=["Sample_ID"]).values.astype(float)

# Keep sample IDs and variable names as separate arrays for labelling plots
sample_ids = df_spectra["Sample_ID"].values
variable_names = df_spectra.drop(columns=["Sample_ID"]).columns.values

# --- Determine x-axis type ---
# If variable names can be converted to numbers, they represent wavenumbers (FTIR)
# or chemical shifts (NMR). Otherwise, use a generic variable index.
try:
    x_axis = np.array([float(v) for v in variable_names])
    # FTIR convention: wavenumbers decrease left-to-right (4000 -> 400)
    # NMR convention: chemical shifts increase left-to-right
    x_label = "Wavenumber (cm⁻¹)" if x_axis[0] > x_axis[-1] else "Chemical shift (ppm)"
except ValueError:
    x_axis = np.arange(X_raw.shape[1])
    x_label = "Variable index"

# --- Merge class labels ---
# Join metadata to spectral data on Sample_ID to ensure correct alignment
merged = df_spectra[["Sample_ID"]].merge(df_metadata, on="Sample_ID", how="left")
class_labels = merged["Class"].values
unique_classes = np.unique(class_labels)

# --- Summary ---
print(f"Spectral matrix X: {X_raw.shape[0]} samples × {X_raw.shape[1]} variables")
print(f"Classes: {list(unique_classes)}")
print(f"X-axis: {x_label}, range {x_axis.min():.1f} – {x_axis.max():.1f}")

In [ ]:
# ============================================================
# INTERACTIVE RAW SPECTRA PLOT
# ============================================================
# Plot all raw spectra colour-coded by class using Plotly.
# Hover over any line to see the sample ID — useful for spotting outliers.

fig = go.Figure()

# --- Colour palette ---
# Use Plotly's Set2 qualitative palette (colour-blind friendly)
colours = px.colors.qualitative.Set2

# Also create hex versions for matplotlib compatibility (used later in HCA)
def plotly_rgb_to_hex(rgb_str):
    """Convert Plotly 'rgb(r,g,b)' string to matplotlib-compatible '#rrggbb'."""
    vals = rgb_str.replace("rgb(", "").replace(")", "").split(",")
    return "#{:02x}{:02x}{:02x}".format(int(vals[0]), int(vals[1]), int(vals[2]))

hex_colours = [plotly_rgb_to_hex(c) if c.startswith("rgb") else c for c in colours]

# --- Plot each class as a group ---
for i, cls in enumerate(unique_classes):
    mask = class_labels == cls  # Boolean mask for samples of this class
    for j, idx in enumerate(np.where(mask)[0]):
        fig.add_trace(go.Scatter(
            x=x_axis,
            y=X_raw[idx],
            name=cls if j == 0 else None,          # Only show class name once in legend
            legendgroup=cls,                        # Group all traces of same class
            showlegend=(j == 0),
            line=dict(color=colours[i % len(colours)], width=0.8),
            hovertext=sample_ids[idx],              # Show sample ID on hover
            hoverinfo="text+y",
            opacity=0.7
        ))

# Reverse x-axis for FTIR (conventional display: high -> low wavenumber)
if x_axis[0] > x_axis[-1]:
    fig.update_xaxes(autorange="reversed")

fig.update_layout(
    title="Raw Spectra by Class",
    xaxis_title=x_label,
    yaxis_title="Absorbance / Intensity",
    template="plotly_white",
    height=500,
    hovermode="closest"
)
fig.show()

In [ ]:
# ============================================================
# SUMMARY STATISTICS & MISSING VALUES
# ============================================================
# Basic data quality checks before analysis. Look for:
# - Missing/infinite values (would break PCA)
# - Unusual intensity ranges (suggests scaling issues)
# - Large differences between class means (good sign for discrimination)

print("=== Data Quality Check ===")
print(f"Missing values: {np.isnan(X_raw).sum()}")
print(f"Infinite values: {np.isinf(X_raw).sum()}")
print(f"\nIntensity range: {X_raw.min():.4f} to {X_raw.max():.4f}")
print(f"Mean intensity:  {X_raw.mean():.4f} ± {X_raw.std():.4f}")

# Per-class summary — large differences here suggest the classes are spectrally distinct
print("\n=== Per-Class Mean Intensity ===")
for cls in unique_classes:
    mask = class_labels == cls
    print(f"  {cls}: {X_raw[mask].mean():.4f} ± {X_raw[mask].std():.4f} (n={mask.sum()})")

## 5. Spectral Preprocessing

Raw spectra typically contain artefacts that must be removed before multivariate analysis:

- **Baseline drift** — gradual shift in the spectrum baseline due to scattering or instrument effects. Corrected using asymmetric least squares (ALS) smoothing or polynomial fitting.
- **Intensity variation** — differences in overall signal intensity between samples due to pathlength or concentration differences. Corrected by normalisation (SNV, min-max, or area normalisation).
- **Noise and band overlap** — high-frequency noise or overlapping peaks can be resolved using Savitzky-Golay derivatives, which also remove constant and linear baseline offsets.

> **Key principle:** Always visualise the effect of each preprocessing step. Over-processing can destroy genuine chemical information.

In [ ]:
# ============================================================
# PREPROCESSING FUNCTIONS
# ============================================================
# These functions implement standard spectral preprocessing methods.
# Each can be applied independently or in combination.

def baseline_als(y, lam=1e6, p=0.01, niter=10):
    """
    Asymmetric Least Squares baseline correction (Eilers & Boelens, 2005).
    
    Fits a smooth curve that follows the lower envelope of the spectrum.
    The baseline is then subtracted to remove drift.
    
    Parameters:
        y     : 1D array — single spectrum
        lam   : float — smoothness parameter. Larger values give a smoother
                baseline. Typical range: 1e4 (flexible) to 1e8 (very smooth).
        p     : float — asymmetry parameter. Smaller values force the baseline
                closer to the lower envelope. Typical range: 0.001 to 0.05.
        niter : int — number of reweighting iterations (10 is usually sufficient)
    
    Returns:
        1D array — the estimated baseline (subtract from y to correct)
    """
    L = len(y)
    # Second-difference matrix penalises curvature in the baseline
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    w = np.ones(L)  # Initial weights (uniform)
    for _ in range(niter):
        W = sparse.spdiags(w, 0, L, L)       # Diagonal weight matrix
        Z = W + lam * D.dot(D.transpose())    # Penalised least squares system
        z = sparse.linalg.spsolve(Z, w * y)   # Solve for baseline
        # Asymmetric reweighting: points above baseline get weight p,
        # points below get weight (1-p). Since p << 1, the baseline
        # is pulled toward the lower envelope.
        w = p * (y > z) + (1 - p) * (y < z)
    return z


def snv(X):
    """
    Standard Normal Variate (SNV) normalisation.
    
    Each spectrum is independently centred (subtract its mean) and
    scaled (divide by its standard deviation). This removes multiplicative
    and additive scatter effects that vary between samples.
    
    Parameters:
        X : 2D array (samples x variables) — spectral matrix
    
    Returns:
        2D array — SNV-normalised spectra
    """
    X_snv = np.zeros_like(X)
    for i in range(X.shape[0]):
        X_snv[i] = (X[i] - np.mean(X[i])) / np.std(X[i])
    return X_snv


def area_normalise(X):
    """
    Area normalisation: scale each spectrum so its total absolute area = 1.
    
    Commonly used for NMR data where total signal intensity should be
    proportional to concentration. Not ideal for FTIR where negative
    absorbance values can occur after baseline correction.
    
    Parameters:
        X : 2D array (samples x variables)
    
    Returns:
        2D array — area-normalised spectra
    """
    return X / np.abs(X).sum(axis=1, keepdims=True)


def minmax_normalise(X):
    """
    Min-max normalisation: scale each spectrum to the [0, 1] range.
    
    Simple but can amplify noise in low-intensity spectra.
    
    Parameters:
        X : 2D array (samples x variables)
    
    Returns:
        2D array — min-max normalised spectra
    """
    mins = X.min(axis=1, keepdims=True)
    maxs = X.max(axis=1, keepdims=True)
    return (X - mins) / (maxs - mins)


def savgol_derivative(X, window_length=15, polyorder=2, deriv=1):
    """
    Savitzky-Golay derivative filter.
    
    Smooths the spectrum with a local polynomial fit, then computes the
    derivative analytically. Benefits:
    - 1st derivative removes constant baseline offsets
    - 2nd derivative removes linear baseline slopes
    - Both enhance resolution of overlapping peaks
    
    Parameters:
        X             : 2D array (samples x variables)
        window_length : int (must be odd) — larger = more smoothing
        polyorder     : int — polynomial degree (2 or 3 typical)
        deriv         : int — derivative order (1 = first, 2 = second)
    
    Returns:
        2D array — derivative spectra
    """
    return signal.savgol_filter(X, window_length, polyorder, deriv=deriv, axis=1)


print("✅ Preprocessing functions defined.")

### 5.1 Apply Preprocessing

Modify the settings below to experiment with different preprocessing combinations. A typical workflow for FTIR data is: **baseline correction → SNV → (optional) 1st derivative**.

For NMR data, you might skip baseline correction and use: **area normalisation → (optional) binning**.

In [ ]:
# ============================================================
# PREPROCESSING SETTINGS — MODIFY THESE
# ============================================================
# Change these parameters and re-run this cell to see the effect.
# The preprocessed data (X_processed) is used by all subsequent analyses.

# Step 1: Baseline correction (ALS)
APPLY_BASELINE = True           # Set to False to skip
BASELINE_LAMBDA = 1e6           # Smoothness: try 1e4 (flexible) to 1e8 (rigid)
BASELINE_P = 0.01               # Asymmetry: try 0.001 (tight fit) to 0.05 (loose)

# Step 2: Normalisation
NORMALISATION = "snv"            # Options: "snv", "area", "minmax", "none"

# Step 3: Savitzky-Golay derivative
APPLY_DERIVATIVE = False         # Set to True to apply
SG_WINDOW = 15                   # Window length (must be odd, larger = smoother)
SG_POLYORDER = 2                 # Polynomial order (2 or 3)
SG_DERIV = 1                     # Derivative order: 1 (removes offsets) or 2 (removes slopes)

# ============================================================
# APPLY PREPROCESSING PIPELINE
# ============================================================
# Each step transforms X_processed in place. The order matters:
# baseline correction first (removes drift), then normalisation
# (removes intensity variation), then optionally derivatives.

X_processed = X_raw.copy()  # Start from raw data each time

# Step 1: Baseline correction
if APPLY_BASELINE:
    print("Applying baseline correction (ALS)...")
    for i in range(X_processed.shape[0]):
        bl = baseline_als(X_processed[i], lam=BASELINE_LAMBDA, p=BASELINE_P)
        X_processed[i] = X_processed[i] - bl  # Subtract estimated baseline

# Step 2: Normalisation
if NORMALISATION == "snv":
    print("Applying SNV normalisation...")
    X_processed = snv(X_processed)
elif NORMALISATION == "area":
    print("Applying area normalisation...")
    X_processed = area_normalise(X_processed)
elif NORMALISATION == "minmax":
    print("Applying min-max normalisation...")
    X_processed = minmax_normalise(X_processed)
else:
    print("No normalisation applied.")

# Step 3: Derivative
if APPLY_DERIVATIVE:
    deriv_suffix = "st" if SG_DERIV == 1 else "nd"
    print(f"Applying Savitzky-Golay {SG_DERIV}{deriv_suffix} derivative...")
    X_processed = savgol_derivative(X_processed, SG_WINDOW, SG_POLYORDER, SG_DERIV)

print(f"\n✅ Preprocessing complete. Matrix shape: {X_processed.shape}")

In [ ]:
# ============================================================
# VISUALISE PREPROCESSING EFFECT
# ============================================================
# Side-by-side comparison of raw vs preprocessed spectra.
# This is essential for quality control — check that:
# - Baseline drift has been removed (spectra should be flat outside peaks)
# - Normalisation has equalised overall intensity
# - No genuine peaks have been destroyed

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Before preprocessing", "After preprocessing"),
    shared_xaxes=True,        # Same x-axis for easy comparison
    vertical_spacing=0.08
)

for i, cls in enumerate(unique_classes):
    mask = class_labels == cls
    for j, idx in enumerate(np.where(mask)[0]):
        # Top panel: raw spectra
        fig.add_trace(go.Scatter(
            x=x_axis, y=X_raw[idx], name=cls if j == 0 else None,
            legendgroup=cls, showlegend=(j == 0),
            line=dict(color=colours[i % len(colours)], width=0.6), opacity=0.6
        ), row=1, col=1)
        # Bottom panel: preprocessed spectra
        fig.add_trace(go.Scatter(
            x=x_axis, y=X_processed[idx], name=cls if j == 0 else None,
            legendgroup=cls, showlegend=False,
            line=dict(color=colours[i % len(colours)], width=0.6), opacity=0.6
        ), row=2, col=1)

# Reverse x-axis for FTIR convention
if x_axis[0] > x_axis[-1]:
    fig.update_xaxes(autorange="reversed")

fig.update_layout(title="Preprocessing Comparison", template="plotly_white", height=700)
fig.update_xaxes(title_text=x_label, row=2, col=1)
fig.update_yaxes(title_text="Absorbance", row=1, col=1)
fig.update_yaxes(title_text="Preprocessed", row=2, col=1)
fig.show()

## 6. Principal Component Analysis (PCA)

PCA is an **unsupervised** method that reduces the dimensionality of spectral data by finding directions (principal components) of maximum variance. It is typically the first step in chemometric analysis because it:

- Reveals natural groupings (or lack thereof) in the data
- Identifies outliers
- Shows which spectral regions contribute most to variation (via loadings)
- Helps determine how many latent variables are needed for supervised models

> **Important:** PCA does not use class labels — any separation seen in the scores plot reflects genuine chemical differences captured by the spectral data.

In [ ]:
# ============================================================
# PCA — FIT AND EXPLORE
# ============================================================
N_COMPONENTS = 10  # Number of PCs to compute (more than needed, we'll inspect the scree plot)

# --- Mean-centring ---
# Standard practice for PCA on spectral data: subtract the mean spectrum.
# We do NOT scale by standard deviation (with_std=False) because all
# variables share the same unit (absorbance) and scaling would distort
# the spectral shape.
scaler = StandardScaler(with_std=False)
X_mc = scaler.fit_transform(X_processed)

# --- Fit PCA ---
# n_components is limited to min(n_samples, n_variables) by definition
pca = PCA(n_components=min(N_COMPONENTS, X_mc.shape[0], X_mc.shape[1]))
scores = pca.fit_transform(X_mc)       # Project data into PC space (n_samples x n_PCs)
loadings = pca.components_              # PC loadings (n_PCs x n_variables)
explained_var = pca.explained_variance_ratio_ * 100  # Percentage variance per PC

# --- Report explained variance ---
print(f"PCA fitted with {pca.n_components_} components.")
print(f"\nExplained variance per PC:")
for i, ev in enumerate(explained_var):
    cumulative = explained_var[:i+1].sum()
    bar = "█" * int(ev)  # Visual bar proportional to variance
    print(f"  PC{i+1:2d}: {ev:5.1f}% {bar}  (cumulative: {cumulative:.1f}%)")

In [ ]:
# ============================================================
# SCREE PLOT
# ============================================================
# The scree plot shows how much variance each PC captures.
# Use it to decide how many PCs are "meaningful":
# - Look for an "elbow" where the curve flattens
# - Typically, PCs beyond the elbow capture mostly noise

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar chart: individual variance per PC
fig.add_trace(go.Bar(
    x=[f"PC{i+1}" for i in range(len(explained_var))],
    y=explained_var,
    name="Individual",
    marker_color="steelblue"
), secondary_y=False)

# Line chart: cumulative variance
fig.add_trace(go.Scatter(
    x=[f"PC{i+1}" for i in range(len(explained_var))],
    y=np.cumsum(explained_var),
    name="Cumulative",
    mode="lines+markers",
    marker=dict(color="firebrick"),
    line=dict(color="firebrick")
), secondary_y=True)

fig.update_layout(title="Scree Plot", template="plotly_white", height=400)
fig.update_yaxes(title_text="Explained variance (%)", secondary_y=False)
fig.update_yaxes(title_text="Cumulative (%)", secondary_y=True, range=[0, 105])
fig.show()

In [ ]:
# ============================================================
# PCA SCORES PLOT (interactive with 95% confidence ellipses)
# ============================================================
# The scores plot shows each sample as a point in PC space.
# Samples that are spectrally similar cluster together.
# Change PC_X and PC_Y to explore different PC combinations.

PC_X = 1  # Which PC on x-axis (1-indexed, so PC1 = first component)
PC_Y = 2  # Which PC on y-axis

# Build a DataFrame for Plotly Express (needs tidy data format)
scores_df = pd.DataFrame({
    "Sample_ID": sample_ids,
    "Class": class_labels,
    f"PC{PC_X}": scores[:, PC_X - 1],
    f"PC{PC_Y}": scores[:, PC_Y - 1]
})

fig = px.scatter(
    scores_df, x=f"PC{PC_X}", y=f"PC{PC_Y}",
    color="Class", hover_name="Sample_ID",
    title=f"PCA Scores Plot: PC{PC_X} vs PC{PC_Y}",
    labels={
        f"PC{PC_X}": f"PC{PC_X} ({explained_var[PC_X-1]:.1f}%)",
        f"PC{PC_Y}": f"PC{PC_Y} ({explained_var[PC_Y-1]:.1f}%)"
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)

# --- Add 95% confidence ellipses ---
# These show the region where 95% of samples from each class are expected
# to fall, assuming a bivariate normal distribution. Calculated from the
# eigenvalues/vectors of the 2x2 covariance matrix and the chi-squared
# critical value for 2 degrees of freedom (5.991 at 95%).
for cls in unique_classes:
    mask = class_labels == cls
    pc_x = scores[mask, PC_X - 1]
    pc_y = scores[mask, PC_Y - 1]
    
    if len(pc_x) < 3:  # Need at least 3 points for a covariance matrix
        continue
    
    # Covariance matrix of the two PCs for this class
    cov = np.cov(pc_x, pc_y)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    angle = np.degrees(np.arctan2(eigenvectors[1, 1], eigenvectors[0, 1]))
    chi2_val = 5.991  # 95% confidence, 2 degrees of freedom
    
    # Generate ellipse boundary points
    theta = np.linspace(0, 2 * np.pi, 100)
    a = np.sqrt(eigenvalues[1] * chi2_val)  # Semi-major axis
    b = np.sqrt(eigenvalues[0] * chi2_val)  # Semi-minor axis
    ellipse_x = a * np.cos(theta)
    ellipse_y = b * np.sin(theta)
    
    # Rotate ellipse to align with eigenvectors, then translate to class centre
    cos_a, sin_a = np.cos(np.radians(angle)), np.sin(np.radians(angle))
    rot_x = cos_a * ellipse_x - sin_a * ellipse_y + pc_x.mean()
    rot_y = sin_a * ellipse_x + cos_a * ellipse_y + pc_y.mean()
    
    fig.add_trace(go.Scatter(
        x=rot_x, y=rot_y, mode="lines",
        line=dict(dash="dash", width=1),
        showlegend=False, hoverinfo="skip"
    ))

fig.update_layout(template="plotly_white", height=550, width=700)
fig.show()

In [ ]:
# ============================================================
# PCA LOADINGS PLOT
# ============================================================
# Loadings show which spectral variables (wavenumbers) contribute
# most to each PC. Peaks in the loadings correspond to absorption
# bands that drive the separation seen in the scores plot.
#
# Positive loadings: variables that increase the PC score
# Negative loadings: variables that decrease the PC score

LOADING_PC = 1  # Which PC loadings to plot (1-indexed)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_axis, y=loadings[LOADING_PC - 1],
    mode="lines", name=f"PC{LOADING_PC} loadings",
    line=dict(color="steelblue")
))

# Zero line for reference
fig.add_hline(y=0, line_dash="dash", line_color="grey")

if x_axis[0] > x_axis[-1]:
    fig.update_xaxes(autorange="reversed")

fig.update_layout(
    title=f"PC{LOADING_PC} Loadings ({explained_var[LOADING_PC-1]:.1f}% variance)",
    xaxis_title=x_label,
    yaxis_title="Loading",
    template="plotly_white",
    height=400
)
fig.show()

# --- Report top contributing variables ---
# Sort by absolute loading value to find the most influential wavenumbers
print(f"\nTop contributing variables for PC{LOADING_PC}:")
abs_loadings = np.abs(loadings[LOADING_PC - 1])
top_idx = np.argsort(abs_loadings)[::-1][:10]
for idx in top_idx:
    print(f"  {variable_names[idx]:>10s}: {loadings[LOADING_PC-1][idx]:+.4f}")

### 6.1 Hotelling's T² — Outlier Detection

Hotelling's T² statistic identifies samples that are unusually far from the centre of the PCA model. Samples exceeding the 95% or 99% confidence limit may be outliers that distort subsequent analysis.

> **Action:** If outliers are identified, investigate them. They may represent measurement errors, contamination, or genuinely unusual samples. Remove them only with justification.

In [ ]:
# ============================================================
# HOTELLING'S T² OUTLIER DETECTION
# ============================================================
# Hotelling's T² is the multivariate equivalent of a z-score.
# It measures how far each sample is from the PCA model centre,
# accounting for the correlation structure between PCs.
#
# T² = score_vector @ inverse_covariance_matrix @ score_vector.T
#
# Critical values come from the F-distribution, scaled for sample size.

N_PC_HOTELLING = 3  # Number of PCs to include (usually 2-5)

# Extract scores for the selected PCs
scores_subset = scores[:, :N_PC_HOTELLING]

# Inverse covariance matrix of the scores
cov_inv = np.linalg.inv(np.cov(scores_subset.T))

# Compute T² for each sample
T2 = np.array([s @ cov_inv @ s.T for s in scores_subset])

# --- Critical values from F-distribution ---
# Formula: T²_crit = p(n-1)(n+1) / n(n-p) * F_crit(p, n-p)
# where p = number of PCs, n = number of samples
n = X_mc.shape[0]
p = N_PC_HOTELLING
T2_95 = (p * (n - 1) * (n + 1)) / (n * (n - p)) * f_dist.ppf(0.95, p, n - p)
T2_99 = (p * (n - 1) * (n + 1)) / (n * (n - p)) * f_dist.ppf(0.99, p, n - p)

# --- Plot ---
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(n)), y=T2,
    mode="markers+text",
    text=sample_ids,
    textposition="top center",
    textfont=dict(size=8),
    marker=dict(
        # Colour each point by its class (using hex colours for compatibility)
        color=[hex_colours[list(unique_classes).index(c) % len(hex_colours)] for c in class_labels],
        size=8
    ),
    hovertext=sample_ids,
    name="Samples"
))

# Confidence limit lines
fig.add_hline(y=T2_95, line_dash="dash", line_color="orange",
              annotation_text="95% limit", annotation_position="top right")
fig.add_hline(y=T2_99, line_dash="dash", line_color="red",
              annotation_text="99% limit", annotation_position="top right")

fig.update_layout(
    title=f"Hotelling's T² (based on {N_PC_HOTELLING} PCs)",
    xaxis_title="Sample index",
    yaxis_title="T²",
    template="plotly_white",
    height=400
)
fig.show()

# --- Report outliers ---
outliers_95 = sample_ids[T2 > T2_95]
outliers_99 = sample_ids[T2 > T2_99]
print(f"Samples exceeding 95% limit: {list(outliers_95) if len(outliers_95) > 0 else 'None'}")
print(f"Samples exceeding 99% limit: {list(outliers_99) if len(outliers_99) > 0 else 'None'}")

## 7. Hierarchical Cluster Analysis (HCA)

HCA is another **unsupervised** method that groups samples based on their similarity. Unlike PCA, it produces a **dendrogram** — a tree-like diagram showing how samples cluster together. This is useful for:

- Confirming groupings seen in PCA
- Identifying sub-groups within a class
- Assessing how well different classes separate

**Key parameters:**
- **Distance metric** — how similarity is measured (Euclidean, correlation, cosine)
- **Linkage method** — how clusters are merged (Ward's is generally recommended for spectral data)

In [ ]:
# ============================================================
# HIERARCHICAL CLUSTER ANALYSIS
# ============================================================
DISTANCE_METRIC = "euclidean"   # Options: "euclidean", "correlation", "cosine"
LINKAGE_METHOD = "ward"          # Options: "ward", "complete", "average", "single"
# Note: Ward's method minimises within-cluster variance and requires Euclidean distance.

# --- Compute linkage matrix ---
# Z encodes the dendrogram structure: each row is a merge step
# [cluster_1, cluster_2, distance, n_samples_in_new_cluster]
Z = linkage(X_processed, method=LINKAGE_METHOD, metric=DISTANCE_METRIC)

# --- Colour map for class labels ---
# Uses hex colours (converted from Plotly palette) for matplotlib compatibility
class_colour_map = {cls: hex_colours[i % len(hex_colours)] for i, cls in enumerate(unique_classes)}

# --- Plot dendrogram ---
fig, ax = plt.subplots(figsize=(14, 5))
dend = dendrogram(
    Z,
    labels=sample_ids,       # Label each leaf with sample ID
    leaf_rotation=90,         # Rotate labels for readability
    leaf_font_size=8,
    ax=ax,
    color_threshold=0         # Colour all links the same (we colour labels instead)
)

# --- Colour leaf labels by class ---
# This makes it easy to see whether the dendrogram correctly groups samples by class
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    sample = lbl.get_text()
    idx = list(sample_ids).index(sample)
    lbl.set_color(class_colour_map[class_labels[idx]])

ax.set_title(f"HCA Dendrogram ({LINKAGE_METHOD} linkage, {DISTANCE_METRIC} distance)")
ax.set_ylabel("Distance")

# Add a legend mapping colours to classes
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=class_colour_map[c], label=c) for c in unique_classes]
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.show()

## 8. Partial Least Squares Discriminant Analysis (PLS-DA)

PLS-DA is a **supervised** method that finds latent variables maximising the covariance between the spectral data (X) and the class labels (Y). Unlike PCA, it uses class information to find the most discriminating directions in spectral space.

**Key considerations:**
- The number of latent variables (LVs) must be optimised by cross-validation to avoid overfitting
- **VIP scores** (Variable Importance in Projection) identify which spectral regions drive classification
- Always validate with cross-validation — never rely on calibration performance alone

> **Overfitting warning:** PLS-DA can easily overfit, especially with many more variables than samples (the typical case for spectral data). Cross-validation is essential.

In [ ]:
# ============================================================
# PLS-DA: ENCODE CLASSES AND SELECT NUMBER OF LVs
# ============================================================
MAX_LV = 10  # Maximum number of latent variables to test

# --- Encode class labels ---
# PLS regression requires numeric Y. For multiclass problems, we use
# a dummy (one-hot) encoding: each class becomes a column with 0/1 values.
# Example for 3 classes: EVOO=[1,0,0], ROO=[0,1,0], Adulterated=[0,0,1]
le = LabelEncoder()
y_encoded = le.fit_transform(class_labels)  # Integer encoding: 0, 1, 2

n_classes = len(unique_classes)
Y_dummy = np.zeros((len(y_encoded), n_classes))
for i, y in enumerate(y_encoded):
    Y_dummy[i, y] = 1  # Set the column corresponding to this sample's class to 1

# --- Cross-validation to select optimal number of LVs ---
# We use 5-fold stratified CV: data is split into 5 parts, each class
# represented proportionally in each fold. The model is trained on 4 folds
# and tested on the held-out fold, rotated 5 times.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=11088)
cv_scores = []

for n_lv in range(1, MAX_LV + 1):
    pls = PLSRegression(n_components=n_lv, scale=True)
    # cross_val_predict returns out-of-fold predictions for every sample
    y_pred_cv = cross_val_predict(pls, X_processed, Y_dummy, cv=cv)
    # Convert continuous PLS predictions back to class labels
    # by selecting the column with the highest predicted value
    y_pred_class = le.inverse_transform(np.argmax(y_pred_cv, axis=1))
    acc = accuracy_score(class_labels, y_pred_class)
    cv_scores.append(acc)

# The optimal number of LVs is the one with highest CV accuracy
optimal_lv = np.argmax(cv_scores) + 1

# --- Plot CV accuracy vs number of LVs ---
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, MAX_LV + 1)), y=cv_scores,
    mode="lines+markers", marker=dict(color="steelblue")
))
fig.add_vline(x=optimal_lv, line_dash="dash", line_color="red",
              annotation_text=f"Optimal: {optimal_lv} LVs")

fig.update_layout(
    title="PLS-DA: Cross-Validation Accuracy vs Number of LVs",
    xaxis_title="Number of Latent Variables",
    yaxis_title="CV Accuracy",
    template="plotly_white",
    height=400,
    yaxis=dict(range=[0, 1.05])
)
fig.show()

print(f"\nOptimal number of LVs: {optimal_lv} (CV accuracy: {cv_scores[optimal_lv-1]:.3f})")

In [ ]:
# ============================================================
# PLS-DA: FIT FINAL MODEL AND EVALUATE
# ============================================================
N_LV = optimal_lv  # Use the optimal value, or override manually here

# Fit PLS-DA on the full dataset with the selected number of LVs
pls_final = PLSRegression(n_components=N_LV, scale=True)
pls_final.fit(X_processed, Y_dummy)

# --- Cross-validated predictions ---
# These are out-of-fold predictions (each sample predicted by a model
# that did NOT see it during training), giving an honest accuracy estimate
y_pred_cv = cross_val_predict(pls_final, X_processed, Y_dummy, cv=cv)
y_pred_class = le.inverse_transform(np.argmax(y_pred_cv, axis=1))

# --- Confusion matrix ---
# Rows = true class, columns = predicted class
# Perfect classification has all counts on the diagonal
cm = confusion_matrix(class_labels, y_pred_class, labels=le.classes_)
fig = px.imshow(
    cm, x=le.classes_, y=le.classes_,
    labels=dict(x="Predicted", y="True", color="Count"),
    text_auto=True,                     # Show counts as text in cells
    color_continuous_scale="Blues",
    title=f"PLS-DA Confusion Matrix ({N_LV} LVs, 5-fold CV)"
)
fig.update_layout(height=400, width=500)
fig.show()

# --- Classification report ---
# Precision: of all samples predicted as class X, how many truly are X?
# Recall: of all true class X samples, how many were correctly identified?
# F1-score: harmonic mean of precision and recall
print("\nClassification Report (cross-validated):")
print(classification_report(class_labels, y_pred_class))

In [ ]:
# ============================================================
# PLS-DA SCORES PLOT
# ============================================================
# Similar to PCA scores, but the axes (latent variables) are chosen
# to maximise separation between classes, not just total variance.

pls_scores = pls_final.x_scores_  # Calibration scores (n_samples x n_LVs)

LV_X = 1  # LV for x-axis (1-indexed)
LV_Y = 2  # LV for y-axis

scores_df = pd.DataFrame({
    "Sample_ID": sample_ids,
    "Class": class_labels,
    f"LV{LV_X}": pls_scores[:, LV_X - 1],
    # If only 1 LV was selected, use zeros for the y-axis
    f"LV{LV_Y}": pls_scores[:, LV_Y - 1] if N_LV >= 2 else np.zeros(len(sample_ids))
})

fig = px.scatter(
    scores_df, x=f"LV{LV_X}", y=f"LV{LV_Y}",
    color="Class", hover_name="Sample_ID",
    title=f"PLS-DA Scores Plot: LV{LV_X} vs LV{LV_Y}",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(template="plotly_white", height=550, width=700)
fig.show()

In [ ]:
# ============================================================
# VIP SCORES (Variable Importance in Projection)
# ============================================================
# VIP scores summarise the importance of each spectral variable
# across all PLS components. They answer: "which wavenumbers are
# most important for distinguishing the classes?"
#
# Rule of thumb:
#   VIP > 1.0 : important variable (contributes above average)
#   VIP > 0.8 : moderately important
#   VIP < 0.8 : less important (candidate for removal in variable selection)

def calculate_vip(pls_model, X, Y):
    """
    Calculate VIP scores for a fitted PLS model.
    
    The VIP for variable j is:
        VIP_j = sqrt( p * sum_h( SS_h * w_jh^2 ) / sum_h( SS_h ) )
    
    where p = number of variables, h = component index,
    SS_h = sum of squares of Y explained by component h,
    w_jh = weight of variable j in component h.
    
    Parameters:
        pls_model : fitted PLSRegression object
        X, Y      : the data matrices used to fit the model
    
    Returns:
        1D array of VIP scores (one per variable)
    """
    T = pls_model.x_scores_        # X scores (n_samples x n_components)
    W = pls_model.x_weights_       # X weights (n_variables x n_components)
    Q = pls_model.y_loadings_      # Y loadings (n_targets x n_components)
    
    p_vars = X.shape[1]  # Number of spectral variables
    h = T.shape[1]       # Number of PLS components
    
    # SS_h: variance in Y explained by each component
    s = np.diag(T.T @ T @ Q.T @ Q)
    s_total = s.sum()
    
    # VIP for each variable
    vip = np.zeros(p_vars)
    for i in range(p_vars):
        # Squared normalised weight for variable i across all components,
        # weighted by the Y-variance explained by each component
        weight_sum = np.sum(s * (W[i, :] / np.linalg.norm(W[:, :], axis=0)) ** 2)
        vip[i] = np.sqrt(p_vars * weight_sum / s_total)
    
    return vip

# --- Calculate and plot VIP scores ---
vip_scores = calculate_vip(pls_final, X_processed, Y_dummy)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_axis, y=vip_scores,
    mode="lines", line=dict(color="steelblue"),
    hovertext=[f"{v}: VIP={vip:.2f}" for v, vip in zip(variable_names, vip_scores)]
))

# Importance thresholds
fig.add_hline(y=1.0, line_dash="dash", line_color="red",
              annotation_text="VIP = 1.0 (important)", annotation_position="top right")
fig.add_hline(y=0.8, line_dash="dot", line_color="orange",
              annotation_text="VIP = 0.8", annotation_position="top right")

if x_axis[0] > x_axis[-1]:
    fig.update_xaxes(autorange="reversed")

fig.update_layout(
    title="VIP Scores — Spectral Regions Driving Classification",
    xaxis_title=x_label,
    yaxis_title="VIP Score",
    template="plotly_white",
    height=400
)
fig.show()

# --- Report top VIP variables ---
print(f"\nTop 10 most important spectral variables:")
top_vip_idx = np.argsort(vip_scores)[::-1][:10]
for idx in top_vip_idx:
    print(f"  {variable_names[idx]:>10s} cm⁻¹: VIP = {vip_scores[idx]:.3f}")

## 9. Orthogonal PLS-DA (OPLS-DA)

OPLS-DA (Trygg & Wold, 2002) is an extension of PLS-DA that separates the variation in X into:

- **Predictive** variation — correlated with the class labels (Y)
- **Orthogonal** variation — structured variation unrelated to classes (e.g., batch effects, instrument drift)

This separation has two major advantages:
1. The **predictive scores plot** often shows cleaner class separation than PLS-DA
2. The **S-plot** (covariance vs correlation) identifies reliable biomarkers for class discrimination

> **Note:** OPLS-DA is most effective for **two-class** comparisons. For multiclass problems, consider running pairwise OPLS-DA models.

In [ ]:
# ============================================================
# OPLS-DA IMPLEMENTATION
# ============================================================
# This implements the OPLS algorithm following Trygg & Wold (2002).
# scikit-learn does not include OPLS-DA, so we build it on top of
# PLSRegression, using the orthogonal signal correction (OSC) approach.

class OPLSDA:
    """
    Orthogonal PLS-DA (Trygg & Wold, 2002).
    
    The algorithm works in two stages:
    1. Iteratively identify and remove orthogonal components from X
       (variation in X that is uncorrelated with Y)
    2. Fit a standard PLS model on the filtered X
    
    For two-class problems: 1 predictive + n_ortho orthogonal components.
    
    Attributes after fitting:
        t_pred_     : predictive scores (1D array, n_samples)
        t_ortho_    : orthogonal scores (n_samples x n_ortho)
        p_pred_     : predictive loadings (1D array, n_variables)
        R2Y_        : R-squared for Y (goodness of fit)
        X_filtered_ : X with orthogonal variation removed
    """
    
    def __init__(self, n_ortho=1):
        """
        Parameters:
            n_ortho : int — number of orthogonal components to remove.
                      Usually 1 is sufficient; increase if the predictive
                      scores plot still shows unwanted structure.
        """
        self.n_ortho = n_ortho
    
    def fit(self, X, y):
        """
        Fit OPLS-DA model.
        
        Parameters:
            X : 2D array (n_samples x n_variables) — preprocessed spectra
            y : 1D array (n_samples,) — binary class labels (0 or 1)
        
        Returns:
            self (for method chaining)
        """
        # --- Mean-centre both X and y ---
        self.X_mean_ = X.mean(axis=0)
        self.y_mean_ = y.mean()
        Xc = X - self.X_mean_
        yc = (y - self.y_mean_).reshape(-1, 1)
        
        # Storage for orthogonal components
        self.t_ortho_ = []
        self.p_ortho_ = []
        self.w_ortho_ = []
        
        X_filtered = Xc.copy()
        
        # --- Iteratively extract orthogonal components ---
        for i in range(self.n_ortho):
            # Step 1: Compute PLS weight vector (direction in X most
            # correlated with y). This is the predictive direction.
            w = (X_filtered.T @ yc) / (yc.T @ yc)
            w = w / np.linalg.norm(w)  # Normalise to unit length
            
            # Step 2: PLS scores and loadings for this weight
            t = X_filtered @ w          # Project X onto the weight vector
            p = (X_filtered.T @ t) / (t.T @ t)  # X loadings
            
            # Step 3: Compute orthogonal weight
            # This is the component of p that is perpendicular to w.
            # It captures structured variation in X that is NOT related to y.
            w_ortho = p - (w.T @ p) / (w.T @ w) * w
            w_ortho = w_ortho / np.linalg.norm(w_ortho)
            
            # Step 4: Orthogonal scores and loadings
            t_ortho = X_filtered @ w_ortho
            p_ortho = (X_filtered.T @ t_ortho) / (t_ortho.T @ t_ortho)
            
            # Step 5: Remove the orthogonal component from X
            X_filtered = X_filtered - t_ortho @ p_ortho.T
            
            # Store for later use (transform, S-plot)
            self.t_ortho_.append(t_ortho.ravel())
            self.p_ortho_.append(p_ortho.ravel())
            self.w_ortho_.append(w_ortho.ravel())
        
        # Convert lists to arrays
        self.t_ortho_ = np.array(self.t_ortho_).T   # (n_samples x n_ortho)
        self.p_ortho_ = np.array(self.p_ortho_).T   # (n_variables x n_ortho)
        self.w_ortho_ = np.array(self.w_ortho_).T   # (n_variables x n_ortho)
        
        # --- Fit PLS on filtered (orthogonal-corrected) X ---
        # Now X_filtered contains only predictive + noise variation
        self.pls_ = PLSRegression(n_components=1, scale=False)
        self.pls_.fit(X_filtered, yc)
        
        # Store predictive scores and loadings
        self.t_pred_ = self.pls_.x_scores_.ravel()
        self.p_pred_ = self.pls_.x_loadings_.ravel()
        self.w_pred_ = self.pls_.x_weights_.ravel()
        self.X_filtered_ = X_filtered
        
        # --- Goodness of fit: R²Y ---
        # How much of the variance in y is explained by the predictive component?
        y_pred = self.pls_.predict(X_filtered)
        ss_res = np.sum((yc - y_pred) ** 2)   # Residual sum of squares
        ss_tot = np.sum(yc ** 2)               # Total sum of squares (centred)
        self.R2Y_ = 1 - ss_res / ss_tot
        
        return self
    
    def transform(self, X):
        """Apply orthogonal correction to new data (e.g., test set)."""
        Xc = X - self.X_mean_
        for i in range(self.n_ortho):
            t_ortho = Xc @ self.w_ortho_[:, i:i+1]
            Xc = Xc - t_ortho @ self.p_ortho_[:, i:i+1].T
        return Xc
    
    def predict(self, X):
        """Predict class for new data."""
        X_filt = self.transform(X)
        return self.pls_.predict(X_filt).ravel() + self.y_mean_
    
    def s_plot_data(self):
        """
        Generate S-plot data: covariance (p) vs correlation (pcorr).
        
        The S-plot visualises reliability vs magnitude for each variable:
        - p(cov): the loading value (magnitude of contribution)
        - p(corr): correlation between variable and predictive score (reliability)
        
        Variables in the upper-right or lower-left corners are both
        strongly contributing AND reliably associated with the class difference.
        These are the best biomarker candidates.
        
        Returns:
            p_cov  : 1D array — covariance (loadings)
            p_corr : 1D array — correlation coefficients
        """
        p = self.p_pred_  # Covariance (loadings)
        
        # Pearson correlation between each variable and the predictive score
        t = self.t_pred_
        X_filt = self.X_filtered_
        pcorr = np.array([np.corrcoef(X_filt[:, j], t)[0, 1] for j in range(X_filt.shape[1])])
        
        return p, pcorr


print("✅ OPLS-DA class defined.")

### 9.1 Select Classes and Fit OPLS-DA

OPLS-DA works best for **pairwise** comparisons. Select two classes to compare below. If you have more than two classes, you can repeat this analysis for each pair.

In [ ]:
# ============================================================
# OPLS-DA: SELECT CLASSES AND FIT
# ============================================================
# Choose which two classes to compare. Change these to run different
# pairwise comparisons (e.g., EVOO vs ROO, ROO vs Adulterated).
CLASS_A = unique_classes[0]  # First class (coded as 0)
CLASS_B = unique_classes[2]  # Second class (coded as 1)
N_ORTHO = 1                  # Number of orthogonal components to remove

print(f"Comparing: {CLASS_A} vs {CLASS_B}")

# --- Subset data to the two selected classes ---
mask_ab = np.isin(class_labels, [CLASS_A, CLASS_B])
X_ab = X_processed[mask_ab]
y_ab = (class_labels[mask_ab] == CLASS_B).astype(float)  # Binary: 0 = CLASS_A, 1 = CLASS_B
ids_ab = sample_ids[mask_ab]
labels_ab = class_labels[mask_ab]

print(f"Samples: {(y_ab==0).sum()} x {CLASS_A}, {(y_ab==1).sum()} x {CLASS_B}")

# --- Fit OPLS-DA ---
opls = OPLSDA(n_ortho=N_ORTHO)
opls.fit(X_ab, y_ab)

print(f"\nR²Y = {opls.R2Y_:.3f}")
print(f"Predictive score range: {opls.t_pred_.min():.3f} to {opls.t_pred_.max():.3f}")

In [ ]:
# ============================================================
# OPLS-DA SCORES PLOT: PREDICTIVE vs ORTHOGONAL
# ============================================================
# X-axis: predictive score (separates the two classes)
# Y-axis: orthogonal score (within-class variation unrelated to class)
#
# Good models show clear horizontal separation between classes,
# with the orthogonal axis capturing other structured variation.

opls_df = pd.DataFrame({
    "Sample_ID": ids_ab,
    "Class": labels_ab,
    "t_predictive": opls.t_pred_,
    "t_orthogonal": opls.t_ortho_[:, 0]
})

fig = px.scatter(
    opls_df, x="t_predictive", y="t_orthogonal",
    color="Class", hover_name="Sample_ID",
    title=f"OPLS-DA Scores: {CLASS_A} vs {CLASS_B} (R²Y = {opls.R2Y_:.3f})",
    labels={"t_predictive": "Predictive score (t[1])", "t_orthogonal": "Orthogonal score (t_ortho[1])"},
    color_discrete_sequence=px.colors.qualitative.Set2
)
# Vertical line at zero: samples on the left belong to CLASS_A, right to CLASS_B
fig.add_vline(x=0, line_dash="dash", line_color="grey", opacity=0.5)
fig.update_layout(template="plotly_white", height=550, width=700)
fig.show()

In [ ]:
# ============================================================
# S-PLOT: IDENTIFY RELIABLE BIOMARKERS
# ============================================================
# The S-plot combines two pieces of information for each variable:
#   x-axis: p(cov)  = loading (magnitude of contribution to class difference)
#   y-axis: p(corr) = correlation (reliability of the association)
#
# Variables in the CORNERS of the S-shape are both large in magnitude
# AND highly reliable — these are the best biomarker candidates.
# Upper-right: higher in CLASS_B; Lower-left: higher in CLASS_A

p_cov, p_corr = opls.s_plot_data()

fig = go.Figure()

# All variables as small points, coloured by correlation strength
fig.add_trace(go.Scatter(
    x=p_cov, y=p_corr,
    mode="markers",
    marker=dict(
        size=5,
        color=np.abs(p_corr),           # Colour intensity = reliability
        colorscale="RdYlBu_r",
        showscale=True,
        colorbar=dict(title="|Correlation|")
    ),
    hovertext=[f"{v} cm⁻¹" for v in variable_names],
    hoverinfo="text"
))

# --- Highlight significant variables ---
# Criteria: |p(corr)| > 0.5 AND |p(cov)| in top 10th percentile
p_thresh = np.percentile(np.abs(p_cov), 90)
sig_mask = (np.abs(p_corr) > 0.5) & (np.abs(p_cov) > p_thresh)

if sig_mask.any():
    fig.add_trace(go.Scatter(
        x=p_cov[sig_mask], y=p_corr[sig_mask],
        mode="markers+text",
        text=[f"{v}" for v in variable_names[sig_mask]],
        textposition="top center", textfont=dict(size=8),
        marker=dict(size=10, color="red", symbol="diamond"),
        name="Significant", showlegend=True
    ))

# Reference lines at zero
fig.add_hline(y=0, line_dash="dash", line_color="grey", opacity=0.3)
fig.add_vline(x=0, line_dash="dash", line_color="grey", opacity=0.3)

fig.update_layout(
    title=f"S-Plot: {CLASS_A} vs {CLASS_B}",
    xaxis_title="Covariance p(cov)",
    yaxis_title="Correlation p(corr)",
    template="plotly_white",
    height=550, width=700
)
fig.show()

# --- Report significant variables ---
print("Variables in S-plot corners (reliable biomarkers):")
if sig_mask.any():
    for idx in np.where(sig_mask)[0]:
        direction = "higher in " + CLASS_B if p_cov[idx] > 0 else "higher in " + CLASS_A
        print(f"  {variable_names[idx]:>10s} cm⁻¹: p(cov)={p_cov[idx]:+.4f}, p(corr)={p_corr[idx]:+.3f} ({direction})")
else:
    print("  No variables met both thresholds. Try adjusting the criteria above.")

### 9.2 Permutation Testing (Model Validation)

Permutation testing is the gold standard for validating OPLS-DA models. The procedure:

1. Randomly shuffle the class labels
2. Fit a new OPLS-DA model on the shuffled data
3. Repeat many times (e.g., 100–200 permutations)
4. Compare the real model's R²Y with the distribution of permuted R²Y values

If the real model is significantly better than the permuted models, the classification is genuine and not due to chance or overfitting.

> **Note:** This can take a minute to run depending on the number of permutations.

In [ ]:
# ============================================================
# PERMUTATION TEST FOR OPLS-DA
# ============================================================
# This tests the null hypothesis: "the OPLS-DA model performs no
# better than chance". We repeatedly shuffle class labels, fit
# new models, and compare their R²Y to the real model.

N_PERMUTATIONS = 100  # 100 for quick testing; increase to 200 for publication

print(f"Running {N_PERMUTATIONS} permutations...")

perm_R2Y = []    # R²Y for each permuted model
perm_corr = []   # Correlation between permuted and original y labels

for i in range(N_PERMUTATIONS):
    # Randomly shuffle the class labels (breaks the real X-Y relationship)
    y_perm = np.random.permutation(y_ab)
    
    # Track how correlated the permuted labels are with the original
    # (permutations closer to the original will naturally give higher R²Y)
    corr_with_original = np.abs(np.corrcoef(y_ab, y_perm)[0, 1])
    
    # Fit OPLS-DA on the permuted data
    opls_perm = OPLSDA(n_ortho=N_ORTHO)
    opls_perm.fit(X_ab, y_perm)
    
    perm_R2Y.append(opls_perm.R2Y_)
    perm_corr.append(corr_with_original)
    
    if (i + 1) % 20 == 0:
        print(f"  {i + 1}/{N_PERMUTATIONS} done...")

perm_R2Y = np.array(perm_R2Y)
perm_corr = np.array(perm_corr)

# --- Permutation plot ---
# Each grey dot is one permuted model. The red star is the real model.
# If the real model is well above the permuted cloud, the model is valid.
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=perm_corr, y=perm_R2Y,
    mode="markers", name="Permuted",
    marker=dict(color="lightgrey", size=6, line=dict(color="grey", width=0.5))
))

# Real model at correlation = 1.0 (perfectly correlated with itself)
fig.add_trace(go.Scatter(
    x=[1.0], y=[opls.R2Y_],
    mode="markers", name="Real model",
    marker=dict(color="red", size=14, symbol="star")
))

# Trend line through permuted points (should intercept y-axis near 0)
fit_coef = np.polyfit(perm_corr, perm_R2Y, 1)
x_line = np.linspace(0, 1, 50)
fig.add_trace(go.Scatter(
    x=x_line, y=np.polyval(fit_coef, x_line),
    mode="lines", name="Trend", line=dict(dash="dash", color="grey")
))

fig.update_layout(
    title=f"Permutation Test ({N_PERMUTATIONS} permutations)",
    xaxis_title="Correlation with original y",
    yaxis_title="R²Y",
    template="plotly_white",
    height=500, width=600
)
fig.show()

# --- Statistical significance ---
# p-value = proportion of permuted models with R²Y >= real R²Y
p_value = (perm_R2Y >= opls.R2Y_).sum() / N_PERMUTATIONS
print(f"\nReal model R²Y: {opls.R2Y_:.3f}")
print(f"Permuted R²Y: {perm_R2Y.mean():.3f} ± {perm_R2Y.std():.3f}")
print(f"Permutation p-value: {p_value:.3f}")
if p_value < 0.05:
    print("✅ Model is statistically significant (p < 0.05). Classification is genuine.")
else:
    print("⚠️ Model is NOT significant. Consider overfitting or insufficient class differences.")

## 10. Export Results

Download key results as CSV files for your report.

In [ ]:
# ============================================================
# EXPORT KEY RESULTS
# ============================================================
# Save analysis results as CSV files that can be used in your report
# or imported into other software (Excel, R, SIMCA, etc.)

# --- PCA scores (first 5 components) ---
pca_export = pd.DataFrame(scores[:, :5], columns=[f"PC{i+1}" for i in range(5)])
pca_export.insert(0, "Sample_ID", sample_ids)
pca_export.insert(1, "Class", class_labels)
pca_export.to_csv("pca_scores.csv", index=False)

# --- PCA loadings (first 5 components) ---
loadings_export = pd.DataFrame(loadings[:5].T, columns=[f"PC{i+1}" for i in range(5)])
loadings_export.insert(0, "Variable", variable_names)
loadings_export.to_csv("pca_loadings.csv", index=False)

# --- VIP scores ---
vip_export = pd.DataFrame({"Variable": variable_names, "VIP": vip_scores})
vip_export.to_csv("vip_scores.csv", index=False)

print("✅ Files saved:")
print("  - pca_scores.csv")
print("  - pca_loadings.csv")
print("  - vip_scores.csv")

# --- Download in Colab ---
# The files.download() function triggers a browser download
try:
    from google.colab import files
    files.download("pca_scores.csv")
    files.download("pca_loadings.csv")
    files.download("vip_scores.csv")
except ImportError:
    print("\n(Not running in Colab — files saved to current directory)")

## References and Further Reading

Jolliffe, I.T. (2002). *Principal Component Analysis*, 2nd ed. Springer.

Barker, M. & Rayens, W. (2003). Partial least squares for discrimination. *Journal of Chemometrics*, 17(3), 166–173.

Trygg, J. & Wold, S. (2002). Orthogonal projections to latent structures (O-PLS). *Journal of Chemometrics*, 16(3), 119–128.

Wiklund, S. et al. (2008). Visualization of GC/TOF-MS-based metabolomics data for identification of biochemically interesting compounds using OPLS class models. *Analytical Chemistry*, 80(1), 115–122.

Chong, I.G. & Jun, C.H. (2005). Performance of some variable selection methods when multicollinearity is present. *Chemometrics and Intelligent Laboratory Systems*, 78(1–2), 103–112.

Eilers, P.H.C. & Boelens, H.F.M. (2005). Baseline correction with asymmetric least squares smoothing. Leiden University Medical Centre report.

Barnes, R.J., Dhanoa, M.S. & Lister, S.J. (1989). Standard normal variate transformation and de-trending of near-infrared diffuse reflectance spectra. *Applied Spectroscopy*, 43(5), 772–777.

*Notebook prepared for FBMFOR — Food Fraud Analysis, University of Reading*